## Eigenschaften der Signale

In [ ]:
import os
from main import openFile
from main import getAmplitudes
from main import findBreaks
from main import PlotInterminResults
from main import interpolate
from main import splitAudioArrAtBreaks
from main import rms
import aiBasefrequency
import numpy
from itables import init_notebook_mode
from itables import show
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64



#Module settings
init_notebook_mode(all_interactive=True)
PlotInterminResults(False)


folder = r"../viblib" #Ordner mit .wav Dateien

#############################################################################
# Sucht alle Wav-Dateien im Ordner und gibt deren Namen als Liste zurück
#############################################################################
def loadFileList():
#Lädt json Datei und gibt Inhalt als string zurück
  return {f for f in os.listdir(folder) if f.endswith('.wav')}

#############################################################################
# Sucht alle Stellen an denen das Signal steigt und gibt die gesammtsteigung und die Gesamtdauer der Steigungen zurück
#############################################################################
def increase(arr,tolerance,sr=16000):
   increases =  []
   increases.append([])
   for i, sample in enumerate(arr):
      if i == len(arr)-1:
         break
      if arr[i] < arr[i+1]:
         increases[-1].append(arr[i+1])
      else:
         if len(increases[-1]) > 0:
            increases.append([])
   mList = []
   durations = []
   for increase in increases:
     if len(increase) > 0:
      duration = len(increase)/sr
      amplitudeDiff = increase[-1] - increase[0]
      if amplitudeDiff/duration > tolerance:
        mList.append(amplitudeDiff / duration)
        durations.append(duration)
   
   if len(mList) > 0:
    return numpy.average(mList,weights=durations), sum(durations)
   else: return 0, 0

#############################################################################
# Sucht alle Stellen an denen das Signal sinkt und gibt die gesammtsteigung und die Gesamtdauer der Steigungen zurück
#############################################################################
def decrease(arr,tolerance,sr=16000):
   decreasees =  []
   decreasees.append([])
   for i, sample in enumerate(arr):
      if i == len(arr)-1:
        break
      if arr[i] > arr[i+1]:
         decreasees[-1].append(arr[i+1])
      else:
         if len(decreasees[-1]) > 0:
            decreasees.append([])
   mList = []
   durations = []
   for decrease in decreasees:
     if len(decrease) > 0:
      duration = len(decrease)/sr
      amplitudeDiff = decrease[-1] - decrease[0]
      if amplitudeDiff < -tolerance*duration:
        mList.append(amplitudeDiff / duration)
        durations.append(duration)
   if len(mList) > 0:
    return numpy.average(mList,weights=durations), sum(durations)
   else: return 0, 0

#############################################################################
# Pausen aus Amplituden filtern (Da Amplituden Algorithmus dort nicht funktioniert)
#############################################################################
def filterPause(arr,breaks):
  for b in breaks:
    for i in range(b[0],b[1]):
      arr[i] = 0
  return arr

#############################################################################
# Plot der Signale
#############################################################################
def plot_to_base64(data,y_scale):
    fig, ax = plt.subplots()
    fig.set_size_inches(2,1)
    ax.plot(data)
    ax.set_ylim(y_scale[0],y_scale[1])
    
    # Plot in einen Bytes-Buffer speichern
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    
    # Bild zu base64 kodieren
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    return f'<img src="data:image/png;base64,{img_base64}" height="50"/>'

#############################################################################
# Durchschschnittliche Zeit zwischen zwei Pausen
#############################################################################
def meanSinDuration(breaks,signal_arr,sr):
  if len(breaks) == 0: return len(signal_arr)/sr
  splits = splitAudioArrAtBreaks(signal_arr,breaks)
  return numpy.average([len(split) for split in splits])/sr

#############################################################################
# Signalenergie ohne Pausen
#############################################################################
def signalEnergyWithoutBreaks(breaks,signal_arr):
  if len(breaks) == 0: return rms(signal_arr)
  splits = splitAudioArrAtBreaks(signal_arr,breaks)
  return numpy.average([rms(split) for split in splits])

# Plot der Signale

#############################################################################

signalAnalyses = {}

for name in loadFileList():
    filePath = os.path.join(folder, name)
    signal_arr, sr = openFile(filePath)
    name = os.path.splitext(name)[0]
    signalAnalyses[name] = {}
    
    breaks = findBreaks(signal_arr, sr)
    frequencies, _ = aiBasefrequency.getFrequencies(signal_arr)
    frequencies = aiBasefrequency.outputTooHz(frequencies)
    frequencies = numpy.interp(range(0,len(signal_arr)), numpy.arange(0,len(frequencies)*512,512),frequencies,right=frequencies[-1])
    signalAnalyses[name]["signal"] = plot_to_base64(signal_arr,[-1,1])
    signalAnalyses[name]["frequency"] = plot_to_base64(frequencies,[0,1000])

    rawAmplitudes = getAmplitudes(signal_arr, sr)
    amplitudes = interpolate(rawAmplitudes, len(signal_arr))
    amplitudes = filterPause(amplitudes,breaks)
    signalAnalyses[name]["amplitude"] = plot_to_base64(amplitudes,[0,1])

    signalAnalyses[name]["duration"] = len(signal_arr)/sr

    
    signalAnalyses[name]["breakTotalDuration"] = (numpy.sum([b[1] - b[0] for b in breaks]))/sr
    signalAnalyses[name]["meanBreakDuration"] = signalAnalyses[name]["breakTotalDuration"] / len(breaks)
    signalAnalyses[name]["breaksCount"] = len(breaks)


    incAmplitudes, durationIncAmp = increase(amplitudes,0.02,sr)
    decAmplitudes, durationDecAmp = decrease(amplitudes,0.02,sr)
    incFrequency, durationIncFreq = increase(frequencies,1,sr)
    decFrequency, durationDecFreq = decrease(frequencies,1,sr)
    signalAnalyses[name]["NoFrequencyChangeDuration"] = signalAnalyses[name]["duration"] - (durationIncFreq + durationDecFreq)
    signalAnalyses[name]["NoAmplitudeChangeDuration"] = signalAnalyses[name]["duration"] - (durationIncAmp + durationDecAmp)
    signalAnalyses[name]["NoFrequencyChangeDurationRelative"] = signalAnalyses[name]["NoFrequencyChangeDuration"] / signalAnalyses[name]["duration"] * 100
    signalAnalyses[name]["NoAmplitudeChangeDurationRelative"] = signalAnalyses[name]["NoAmplitudeChangeDuration"] / signalAnalyses[name]["duration"] * 100
    signalAnalyses[name]["meanFrequency"] = numpy.mean(frequencies)

    signalAnalyses[name]["increaseRateFrequency"] = durationIncFreq
    signalAnalyses[name]["decreaseRateFrequency"] = durationDecFreq
    signalAnalyses[name]["increaseRateAmplitude"] = incAmplitudes
    signalAnalyses[name]["decreaseRateAmplitude"] = decAmplitudes
    signalAnalyses[name]["meanTimeBetweenBreaks"] = meanSinDuration(breaks,signal_arr,sr)
    signalAnalyses[name]["meanSignalEnergyWithoutBreaks"] = signalEnergyWithoutBreaks(breaks,signal_arr)
    

## Entfernen dre signal und frequenzspalte
# Spalten, die ausgeschlossen werden sollen:
""" spalten_ausschliessen = ['signal', 'frequency']
# Kopie des Dictionaries erstellen, ohne die unerwünschten Spalten:
signalAnalyses_filtered = {
    key: {k: v for k, v in value.items() if k not in spalten_ausschliessen}
    for key, value in signalAnalyses.items()
} """

signalAnalysesDF = pd.DataFrame.from_dict(signalAnalyses, orient='index')


signalAnalysesDF



## CSV Laden und Daten aufbereiten

In [ ]:
import pandas as pd
import networkx as nx

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

words = {"sensationTags":[],
         "emotionTags":[],
         "metaphors":[],
         "usageExamples":[]}
#Datei öffnen
csv_path = r"C:\Users\Domin\OneDrive\HTW\Bachelorarbeit\WAVTactilTransformer\viblib\vibrationAnnotations-July24th2016.csv"
df = pd.read_csv(csv_path, sep=',', header=0, index_col=0, decimal='.', usecols=["id","sensationTags","emotionTags","metaphors","usageExamples"], dtype={"id":str,"sensationTags":str,"emotionTags":str,"metaphors":str,"usageExamples":str},na_filter=False)

#finde Alarm tags und benenne sie um, da sie in mataphors und usageExamples vorkommen
for index, row in df.iterrows():
    tags = row["usageExamples"].split(",")
    if "alarm" in tags:
        alarmIndex = tags.index("alarm")
        tags[alarmIndex] = "alarm-usageExamples"
        df["usageExamples"][index] = ",".join(tags)
    tags = row["metaphors"].split(",")
    if "alarm" in tags:
        alarmIndex = tags.index("alarm")
        tags[alarmIndex] = "alarm-metaphors"
        df["metaphors"][index] = ",".join(tags)

#alle vorkommenden Tags nach Fasetten aufteilen
heads = ["sensationTags","emotionTags","metaphors","usageExamples"]
for index, row in df.iterrows():
    for head in heads:
        tags=row[head].split(",")
        for tag in tags:
            if tag not in words[head]:
                words[head].append(tag)


#liste mit allen vorkommenden Tags
allTags = words["sensationTags"]+words["emotionTags"]+words["metaphors"]+words["usageExamples"]

## Eigenschaften der Signale pro Tag

In [ ]:
analysePerTag = {}
for category in words:
    for tag in words[category]:
        analysePerTag[tag] = {"category":category 
                              , "IDs" : []}
        
for index, row in df.iterrows():
    tags = (row["sensationTags"]+","+row["emotionTags"]+","+row["metaphors"]+","+row["usageExamples"]).split(",")
    for tag in tags:
        try:
            signalAnalyses[index]
            analysePerTag[tag]["IDs"].append(index)
        except KeyError:
            pass
        
import numpy as np       

for tag in analysePerTag:

    analysePerTag[tag]["meanDuration"] = np.mean([signalAnalyses[ID]["duration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdDuration"] = np.std([signalAnalyses[ID]["duration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["minDuration"] = np.min([signalAnalyses[ID]["duration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["maxDuration"] = np.max([signalAnalyses[ID]["duration"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanTotalBreakDuration"] = np.mean([signalAnalyses[ID]["breakTotalDuration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdTotalBreakDuration"] = np.std([signalAnalyses[ID]["breakTotalDuration"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanBreakCount"] = np.mean([signalAnalyses[ID]["breaksCount"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdBreakCount"] = np.std([signalAnalyses[ID]["breaksCount"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanBreakDuration"] = np.nanmean([signalAnalyses[ID]["meanBreakDuration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdBreakDuration"] = np.nanstd([signalAnalyses[ID]["meanBreakDuration"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanNoFrequencyChangeDuration"] = np.mean([signalAnalyses[ID]["NoFrequencyChangeDuration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdNoFrequencyChangeDuration"] = np.std([signalAnalyses[ID]["NoFrequencyChangeDuration"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanNoAmplitudeChangeDuration"] = np.mean([signalAnalyses[ID]["NoAmplitudeChangeDuration"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdNoAmplitudeChangeDuration"] = np.std([signalAnalyses[ID]["NoAmplitudeChangeDuration"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["MeanNoFrequencyChangeDurationRelative"] = np.mean([signalAnalyses[ID]["NoFrequencyChangeDurationRelative"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdNoFrequencyChangeDurationRelative"] = np.std([signalAnalyses[ID]["NoFrequencyChangeDurationRelative"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["MeanNoAmplitudeChangeDurationRelative"] = np.mean([signalAnalyses[ID]["NoAmplitudeChangeDurationRelative"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdNoAmplitudeChangeDurationRelative"] = np.std([signalAnalyses[ID]["NoAmplitudeChangeDurationRelative"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanIncreaseRateFrequency"] = np.mean([signalAnalyses[ID]["increaseRateFrequency"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdIncreaseRateFrequency"] = np.std([signalAnalyses[ID]["increaseRateFrequency"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanFrequency"] = np.mean([signalAnalyses[ID]["meanFrequency"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdMeanFrequency"] = np.std([signalAnalyses[ID]["meanFrequency"] for ID in analysePerTag[tag]["IDs"]])


    analysePerTag[tag]["meanIncreaseRateAmplitude"] = np.mean([signalAnalyses[ID]["increaseRateAmplitude"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdIncreaseRateAmplitude"] = np.std([signalAnalyses[ID]["increaseRateAmplitude"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanDecreaseRateFrequency"] = np.mean([signalAnalyses[ID]["decreaseRateFrequency"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdDecreaseRateFrequency"] = np.std([signalAnalyses[ID]["decreaseRateFrequency"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanDecreaseRateAmplitude"] = np.mean([signalAnalyses[ID]["decreaseRateAmplitude"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdDecreaseRateAmplitude"] = np.std([signalAnalyses[ID]["decreaseRateAmplitude"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanTimeBetweenBreaks"] = np.mean([signalAnalyses[ID]["meanTimeBetweenBreaks"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdTimeBetweenBreaks"] = np.std([signalAnalyses[ID]["meanTimeBetweenBreaks"] for ID in analysePerTag[tag]["IDs"]])

    analysePerTag[tag]["meanSignalEnergyWithoutBreaks"] = np.mean([signalAnalyses[ID]["meanSignalEnergyWithoutBreaks"] for ID in analysePerTag[tag]["IDs"]])
    analysePerTag[tag]["stdSignalEnergyWithoutBreaks"] = np.std([signalAnalyses[ID]["meanSignalEnergyWithoutBreaks"] for ID in analysePerTag[tag]["IDs"]])


analysePerTagDf=pd.DataFrame.from_dict(analysePerTag,orient="index")
analysePerTagDf



In [ ]:
## Boxplot long short

dataLong = [signalAnalyses[ID]["duration"] for ID in analysePerTag["long"]["IDs"]]
dataShort = [signalAnalyses[ID]["duration"] for ID in analysePerTag["short"]["IDs"]]
plt.title("boxplot der Dauer von \"long\"")
plt.boxplot([signalAnalyses[ID]["duration"] for ID in analysePerTag["long"]["IDs"]])
plt.figure()
plt.title("Box-Plot der Dauer von \"short\"")
plt.boxplot([signalAnalyses[ID]["duration"] for ID in analysePerTag["short"]["IDs"]])
plt.figure()
plt.title("Box-Plot der Signaldauer von vibrotatkilen Mustern mit den Tags \"long\" und \"short\"")
plt.ylabel("Dauer in Sekunden")
plt.boxplot([dataLong, dataShort], labels=["long", "short"])


## Gemeinsames Aufteten von Tags (Absolut)
Je näher die Punkte desto öfter werden die Tags zusammen verwendet

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from IPython.core.display import display 

#Erstelleng des Netzwerkdiagramms
G = nx.Graph()
G.add_nodes_from(words["sensationTags"], type="sensationTags")
G.add_nodes_from(words["emotionTags"], type="emotionTags")
G.add_nodes_from(words["metaphors"], type="metaphors")
G.add_nodes_from(words["usageExamples"], type="usageExamples")

#############################################################
#Berechnung der Kantengewichte bzw anzahl des gemeinsamen Auftretens der Tags (Absolut)
#############################################################


#Dictionary mit Kantengewichten
edges = {}

#Dictionary mit allen Kanten initialisieren
for i in range(len(allTags)):
    edges[allTags[i]] = {}
    for j in range(i+1,len(allTags)):
        edges[allTags[i]][allTags[j]] = 0

#Hilfsfunktion zum Zählen des gemeinsamen Auftretens der Tags
def countEdges(tag1, tag2):
    if tag1 == tag2:
        print(f"Warning: {tag1} and {tag2} are the same")
        return
    try:
        edges[tag1][tag2] += 1
    except:
        edges[tag2][tag1] += 1

#Kanten berechnen
for index, row in df.iterrows():
    line = (row["sensationTags"]+","+row["emotionTags"]+","+row["metaphors"]+","+row["usageExamples"]).split(",")
    for i in range(len(line)):
        for j in range(i+1,len(line)):
            countEdges(line[i], line[j])
#############################################################

#Graph auffüllen mit allen Infromationen
for key in edges:
    for key2 in edges[key]:
        if edges[key][key2] > 0:
            G.add_edge(key, key2, weight=edges[key][key2])

#Ausgabe des Netzwerkdiagramms
color_map = []
for node in G:
    if G.nodes[node]['type'] == "sensationTags":
        color_map.append('red')
    elif G.nodes[node]['type'] == "emotionTags":
        color_map.append('blue')
    elif G.nodes[node]['type'] == "metaphors":
        color_map.append('green')
    elif G.nodes[node]['type'] == "usageExamples":
        color_map.append('purple')
plt.figure(figsize=(15,15), dpi=300)
plt.title("Kookkurrenz Absolut")

# Legende #
legend_labels = {
    'red': 'sensation tags',
    'blue': 'emotion tags',
    'green': 'metaphors',
    'purple': 'usage examples',
}
legend_handles = [mlines.Line2D([], [], color=color, linewidth=2, label=label) 
                  for color, label in legend_labels.items()]
plt.legend(handles=legend_handles)
##

pos = nx.spring_layout(G, weight='weight',k=1.5,iterations=1000,scale=1) 
edge_weights = nx.get_edge_attributes(G, 'weight')
nx.draw(G, pos, with_labels=True, node_color=color_map, edge_color='gray', node_size=300, font_size=4,width = 00.1)

#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_weights)

nx.write_gexf(G, "graphTotal.gexf")
plt.savefig("graphTotal.svg")
plt.savefig("graphTotal.png")

#Adjazenzmatrix
adjTotal =nx.to_pandas_adjacency(G)
adjTotal.to_csv("adj.csv", sep=';', decimal=',')



## Anzahl der Tags

In [ ]:
# Wie oft kommt welches Tag vor?
totalPerTagCount ={}
for tag in allTags:
    totalPerTagCount[tag] = 0

for index, row in df.iterrows():
    tags = (row["sensationTags"]+","+row["emotionTags"]+","+row["metaphors"]+","+row["usageExamples"]).split(",")
    for tag in tags:
        totalPerTagCount[tag] += 1







######

totalPerTagCountCategories = {
        "sensationTags":{},
        "emotionTags":{},
        "metaphors":{},
        "usageExamples":{}}
for category in words:
    for word in words[category]:
        totalPerTagCountCategories[category][word] = 0
for index, row in df.iterrows():
    for tag in row["emotionTags"].split(","):
        totalPerTagCountCategories["emotionTags"][tag] +=1
    for tag in row["metaphors"].split(","):
        totalPerTagCountCategories["metaphors"][tag] +=1
    for tag in row["sensationTags"].split(","):
        totalPerTagCountCategories["sensationTags"][tag] +=1
    for tag in row["usageExamples"].split(","):
        totalPerTagCountCategories["usageExamples"][tag] +=1
    
from IPython.display import HTML
for category in totalPerTagCountCategories:
    display(HTML("<h3>"+category+"</h3>"))
    dfCounts=pd.DataFrame.from_dict(data=totalPerTagCountCategories[category],orient='index',columns=["Anzahl des Tags"])
    dfCounts=dfCounts.sort_values(by="Anzahl des Tags",ascending=False)
    display(dfCounts)
        




## gemeinsames Auftreten von Tags (relativ zum Gesamtvorkommen der Tags)
Je näher die Punkte desto öfter werden die Tags zusammen verwendet

In [ ]:
#Berechnung der relativen Gewichte
#Von Datensätzen in denen mindestens einer von beiden Tags vorkommt. Wie oft kommen sie im Verältnis zusammen vor.
for key in edges:
    for key2 in edges[key]:
        edges[key][key2] = edges[key][key2]  /  (totalPerTagCount[key]+totalPerTagCount[key2]-edges[key][key2]) #(stellen an denen beide Tags vorkommen werden nur einmal gezält)
        
#Erstelleng des Netzwerkdiagramms
G = nx.Graph()
G.add_nodes_from(words["sensationTags"], type="sensationTags")
G.add_nodes_from(words["emotionTags"], type="emotionTags")
G.add_nodes_from(words["metaphors"], type="metaphors")
G.add_nodes_from(words["usageExamples"], type="usageExamples")


#Graph auffüllen mit allen Infromationen
for key in edges:
    for key2 in edges[key]:
        if edges[key][key2] > 0:
            G.add_edge(key, key2, weight=edges[key][key2])

#Ausgabe des Netzwerkdiagramms
color_map = []
for node in G:
    if G.nodes[node]['type'] == "sensationTags":
        color_map.append('red')
    elif G.nodes[node]['type'] == "emotionTags":
        color_map.append('blue')
    elif G.nodes[node]['type'] == "metaphors":
        color_map.append('green')
    elif G.nodes[node]['type'] == "usageExamples":
        color_map.append('purple')
plt.figure(figsize=(15,15), dpi=300)
plt.legend(handles=legend_handles)
plt.title("Kookkurrenz Relativ")
pos = nx.spring_layout(G, weight='weight',k=1,iterations=1000,scale=1) 
edge_weights = nx.get_edge_attributes(G, 'weight')

nx.draw(G, pos, with_labels=True, node_color=color_map, edge_color='gray', node_size=300, font_size=4,width = 00.1)

nx.write_gexf(G, "graphRalativ.gexf")
plt.savefig("graphRelativ.svg")
plt.savefig("graphRekativ.png")
            
#Adjazenzmatrix
adjRelative =nx.to_pandas_adjacency(G)
adjRelative.to_csv("relative.csv",  sep=';', decimal=',')

In [ ]:
wav_path = r"C:\Users\Domin\OneDrive\HTW\Bachelorarbeit\WAVTactilTransformer\viblib"

import matplotlib.pyplot as plt



# Hauptplot erstellen
fig, ax = plt.subplots()
fig.set_size_inches(30,30)
fig.dpi = 300
fig.patch.set_alpha(0)
nx.draw(G, pos, with_labels=True, node_color=color_map, edge_color='gray', node_size=300, font_size=4,width = 00.1)
# ax.set_xticks([])
# ax.set_yticks([])
# ax.patch.set_alpha(0)
# fig2, ax2 = plt.subplots()
# fig2.set_size_inches(30,30)
# fig2.dpi = 300
# fig2.patch.set_alpha(0)
# ax2.set_xticks([])
# ax2.set_yticks([])
# ax2.patch.set_alpha(0)
# fig3, ax3 = plt.subplots()
# fig3.set_size_inches(30,30)
# fig3.dpi = 300
# fig3.patch.set_alpha(0)
# ax3.set_xticks([])
# ax3.set_yticks([])
# ax3.patch.set_alpha(0)
# fig4, ax4 = plt.subplots()
# fig4.set_size_inches(30,30)
# fig4.dpi = 300
# fig4.patch.set_alpha(0)
# ax4.set_xticks([])
# ax4.set_yticks([])
# ax4.patch.set_alpha(0)


#figs = [fig,fig2,fig3,fig4]
figs = [fig]

# #ax.plot([0, 1, 2, 3, 4], [10, 20, 25, 30, 40], label="Hauptdaten")

import os
from main import openFile
def loadWAF(name):
#Lädt json Datei und gibt Inhalt als string zurück
  file = os.path.join(wav_path, name + ".wav")
  if os.path.exists(file):
    vibration_arr , sr = openFile(file)
    return vibration_arr
  else:
    return None

# Schwerpunkt der Vibrationen:
import numpy
focus_points_vibr = {}
for index, row in df.iterrows():
    line = (row["sensationTags"]+","+row["emotionTags"]+","+row["metaphors"]+","+row["usageExamples"]).split(",")
    points=[]
    for tag in line:
        points.append(pos[tag])
    focus_points_vibr[index] = numpy.mean(points, axis = 0)
print(focus_points_vibr)
colors = ["red","green","blue","violet","orange"]
i = 0
for index in focus_points_vibr:
    
    # fig.text(focus_points_vibr[index][0],focus_points_vibr[index][1],index,fontsize=3,color="orange")
    vib_arr = loadWAF(index)
    if vib_arr is None:
        i+=1
        continue
               #[i%4]
    mini2 = figs[0].add_axes([(focus_points_vibr[index][0]+1)/2, (focus_points_vibr[index][1]+1)/2, 0.01 * (len(vib_arr)/16000), 0.01])
    mini2.plot(vib_arr, label="Mini-Daten", color=colors[i%5],linewidth=0.03)
    mini2.set_title(index, fontsize=3)
    mini2.patch.set_alpha(0)
    mini2.set_xticks([])
    mini2.set_yticks([])
    i+=1

for i, fi in enumerate(figs):
   fi.savefig(f"NetzplotTotal_{i}.svg")

## Eigenschaften der Tags Farbkarte

In [ ]:
from scipy.interpolate import griddata
def colored_plot(property1, property2,property3):
    color_propety_map = {}
    scale1=[numpy.min(numpy.nan_to_num([analysePerTag[tag][property1] for tag in analysePerTag])),numpy.max(numpy.nan_to_num([analysePerTag[tag][property1] for tag in analysePerTag]))]
    scale2=[numpy.min(numpy.nan_to_num([analysePerTag[tag][property2] for tag in analysePerTag])),numpy.max(numpy.nan_to_num([analysePerTag[tag][property2] for tag in analysePerTag]))]
    scale3=[numpy.min(numpy.nan_to_num([analysePerTag[tag][property3] for tag in analysePerTag])),numpy.max(numpy.nan_to_num([analysePerTag[tag][property3] for tag in analysePerTag]))]
    
    for tag in pos:
        cValueR = (analysePerTag[tag][property1]-scale1[0])/(scale1[1]-scale1[0])
        cValueG = (analysePerTag[tag][property2]-scale2[0])/(scale2[1]-scale2[0])
        cValueB = (analysePerTag[tag][property3]-scale3[0])/(scale3[1]-scale3[0])
        color_propety_map[tag] = [cValueR, cValueG, cValueB]

        
        #color_propety_map[tag] = [0,0,0]
    

    points = numpy.array(list(pos.values()))
    #points = numpy.array([[-point[1],point[0]] for point in points])
    colors = numpy.array(list(color_propety_map.values()))
    numpy.nan_to_num(colors,copy=False,nan = 1)

    # Bestimme die Extents, damit sie zu den Koordinaten des Graphen passen
    min_x, min_y = points.min(axis=0)
    max_x, max_y = points.max(axis=0)

    grid_x, grid_y = numpy.mgrid[min_y:max_y:100j,min_x:max_x:100j]


    grid_colors = griddata(numpy.array([[-point[1],point[0]] for point in points]), colors, (grid_x, grid_y),method='cubic')

    
    

    plt.imshow(grid_colors, extent=(min_x,max_x, min_y,max_y))
    nx.draw(G, pos, with_labels=True, node_color=colors, edge_color=[0,0,0,0], node_size=300, font_size=4,width = 00.1)  

def colored_plot_one(property1):
    color_propety_map = {}
    scale1=[numpy.min(numpy.nan_to_num([analysePerTag[tag][property1] for tag in analysePerTag])),numpy.max(numpy.nan_to_num([analysePerTag[tag][property1] for tag in analysePerTag]))]

    
    for tag in pos:
        cValueR = (analysePerTag[tag][property1]-scale1[0])/(scale1[1]-scale1[0])

        color_propety_map[tag] = [1, 1-cValueR, 1-cValueR]

        
        #color_propety_map[tag] = [0,0,0]
    

    points = numpy.array(list(pos.values()))
    #points = numpy.array([[-point[1],point[0]] for point in points])
    colors = numpy.array(list(color_propety_map.values()))
    numpy.nan_to_num(colors,copy=False,nan = 1)

    # Bestimme die Extents, damit sie zu den Koordinaten des Graphen passen
    min_x, min_y = points.min(axis=0)
    max_x, max_y = points.max(axis=0)

    grid_x, grid_y = numpy.mgrid[min_y:max_y:100j,min_x:max_x:100j]


    grid_colors = griddata(numpy.array([[-point[1],point[0]] for point in points]), colors, (grid_x, grid_y),method='cubic')

    
    

    plt.imshow(grid_colors, extent=(min_x,max_x, min_y,max_y))
    nx.draw(G, pos, with_labels=True, node_color=colors, edge_color=[0,0,0,0], node_size=300, font_size=4,width = 00.1)   
plt.figure(figsize=(15,15), dpi=300)
""" plt.title("Rot: Anzahl der Pausen, Grün: durchschnittliche Dauer einer Pause, Blau: Durchschnittliche Zeit zwischen Pausen")
colored_plot("meanBreakCount", "meanBreakDuration","meanTimeBetweenBreaks") """
plt.figure(figsize=(15,15), dpi=300)
plt.title("meanDuration")
colored_plot_one("meanDuration")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanSignalEnergyWithoutBreaks")
colored_plot_one("meanSignalEnergyWithoutBreaks")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanTotalBreakDuration")
colored_plot_one("meanTotalBreakDuration")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanTimeBetweenBreaks")
colored_plot_one("meanTimeBetweenBreaks")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanBreakCount")
colored_plot_one("meanBreakCount")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanBreakDuration")
colored_plot_one("meanBreakDuration")


plt.figure(figsize=(15,15), dpi=300)
plt.title("meanNoFrequencyChangeDuration")
colored_plot_one("meanNoFrequencyChangeDuration")

plt.figure(figsize=(15,15), dpi=300)
plt.title("MeanNoFrequencyChangeDurationRelative")
colored_plot_one("MeanNoFrequencyChangeDurationRelative")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanIncreaseRateFrequency")
colored_plot_one("meanIncreaseRateFrequency")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanDecreaseRateFrequency")
colored_plot_one("meanDecreaseRateFrequency")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanFrequency")
colored_plot_one("meanFrequency")


plt.figure(figsize=(15,15), dpi=300)
plt.title("meanNoAmplitudeChangeDuration")
colored_plot_one("meanNoAmplitudeChangeDuration")

plt.figure(figsize=(15,15), dpi=300)
plt.title("MeanNoAmplitudeChangeDurationRelative")
colored_plot_one("MeanNoAmplitudeChangeDurationRelative")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanIncreaseRateAmplitude")
colored_plot_one("meanIncreaseRateAmplitude")

plt.figure(figsize=(15,15), dpi=300)
plt.title("meanDecreaseRateAmplitude")
colored_plot_one("meanDecreaseRateAmplitude")



## Adjazenzmatrix Total

In [ ]:


# Funktion zum Berechnen der Farbe
def color_scale(val):
    # Normierung des Werts zwischen 0 und 1
    normalized_val = val / 40  # Wert liegt zwischen 0 und 1
    
    # Berechnung des Grünanteils basierend auf dem normalisierten Wert
    green_intensity = int(normalized_val * 255)  # Grünanteil berechnen (zwischen 0 und 255)
    
    # Erstellen der Farbe im RGB-Format: rgb(255, 255, 255) -> rgb(255, 255, 255 - green_intensity)
    return f'background-color: rgb(255, 255, {255-green_intensity})'

# Stil anwenden
styled_df = adjTotal.style.map(color_scale)
display(styled_df)

## Adjazenzmatrix relativ

In [ ]:
# Funktion zum Berechnen der Farbe
def color_scale(val):

    
    # Berechnung des Grünanteils basierend auf dem normalisierten Wert
    yellow_intensity = int(val * 255)  # Grünanteil berechnen (zwischen 0 und 255)
    
    # Erstellen der Farbe im RGB-Format: rgb(255, 255, 255) -> rgb(255, 255, 255 - green_intensity)
    return f'background-color: rgb({255-yellow_intensity}, 255, {255-yellow_intensity})'

styled_df = adjRelative.style.map(color_scale)
display(styled_df)